In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../pruebillas.ipynb'), '..'))
os.chdir(root_dir)

sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, plot_heatmaps, resample_apply, barssince
import pandas as pd
import talib as ta
import pandas_ta as pandas_ta
import numpy as np
import matplotlib.pyplot as plt
from backtesting import Strategy
import plotly.express as px
from datetime import datetime
import MetaTrader5 as mt5

import random
random.seed(42)

from backbone.utils.general_purpose import diff_pips

In [3]:
symbols_path = './backbone/data/backtest/symbols/SP500_H1.csv'
df = pd.read_csv(symbols_path)
df

,Date,Open,High,Low,Close,Volume
0,2012-01-16 00:00:00,1290.900,1291.100,1288.100,1288.100,0.0000
1,2012-01-16 01:00:00,1287.900,1288.400,1286.400,1287.400,0.0000
2,2012-01-16 02:00:00,1287.600,1287.600,1286.600,1287.100,0.0000
3,2012-01-16 03:00:00,1286.900,1288.400,1286.900,1287.100,0.0000
4,2012-01-16 04:00:00,1286.900,1287.100,1286.100,1286.100,0.0000
...,...,...,...,...,...,...
63251,2024-08-30 16:00:00,5600.337,5605.937,5581.014,5600.354,29.2410
63252,2024-08-30 17:00:00,5600.116,5607.377,5595.127,5597.849,20.0450
63253,2024-08-30 18:00:00,5597.970,5618.452,5595.220,5616.233,14.5300
63254,2024-08-30 19:00:00,5615.983,5651.087,5610.517,5645.139,23.4720


In [4]:
# from datetime import datetime
# import MetaTrader5 as mt5
# import pytz

# print("MetaTrader5 package author: ", mt5.__author__)
# print("MetaTrader5 package version: ", mt5.__version__)

# # establish connection to MetaTrader 5 terminal
# if not mt5.initialize():
#     raise Exception("initialize() failed, error code =",mt5.last_error())

# # set time zone to UTC
# timezone = pytz.timezone("Etc/UTC")

# # create 'datetime' objects in UTC time zone to avoid the implementation of a local time zone offset
# utc_from = datetime(2013, 5, 1, tzinfo=timezone)
# utc_to = datetime(2024, 1, 1, tzinfo=timezone)
# rates = mt5.copy_rates_range('USDCAD', mt5.TIMEFRAME_H1, utc_from, utc_to)

# mt5.shutdown()

# # create DataFrame out of the obtained data
# df = pd.DataFrame(rates)

# # convert time in seconds into the datetime format
# df['time'] = pd.to_datetime(df['time'], unit='s')
                          
# df = df.rename(columns={
#   'time':'Date', 
#   'open':'Open', 
#   'high':'High', 
#   'low':'Low', 
#   'close':'Close', 
#   'tick_volume':'Volume'
# })

# df

In [5]:
train_start = '2014-01-01'
train_end = '2016-01-01'

test_start = '2015-01-01'
test_end = '2018-01-01'

wfo_start = '2014-01-01'
wfo_end = '2023-01-01'

train_data = df[(df.Date > train_start) & (df.Date < train_end)]
test_data = df[(df.Date > test_start) & (df.Date < test_end)]
wfo_data = df[(df.Date > wfo_start) & (df.Date < wfo_end)]

train_data.loc[:, 'Date'] = pd.to_datetime(train_data.Date)
test_data.loc[:, 'Date'] = pd.to_datetime(test_data.Date)
wfo_data.loc[:, 'Date'] = pd.to_datetime(wfo_data.Date)

df.loc[:, 'Date'] = pd.to_datetime(df.Date)

train_data = train_data.set_index('Date')
test_data = test_data.set_index('Date')
wfo_data = wfo_data.set_index('Date')

df = df.set_index('Date')


In [6]:
def ll_hh_indicator(close, window=None):
    if type(close) != pd.Series:
        close = close.s

    rolling_min = close.rolling(window=window, min_periods=1).min()
    is_lower_low = close == rolling_min

    rolling_max = close.rolling(window=window, min_periods=1).max()
    is_higher_high = close == rolling_max

    return is_lower_low, is_higher_high


In [17]:

import itertools

class LowerLow(Strategy):
    risk=2
    n=10
    
    def init(self):

        self.daily_sma_200 = resample_apply(
            'D', ta.SMA, self.data.Close, timeperiod=200
        )
        
        self.daily_ll, self.daily_hh = resample_apply(
            'D', ll_hh_indicator, self.data.Close, window=self.n
        )
        

    def next(self):
        actual_close = self.data.Close[-1]
        today = self.data.index[-1].tz_localize('UTC').tz_convert('UTC')

        if self.position:
            first_trade = self.trades[0]
            time_in_position = (today - first_trade.entry_time.tz_localize('UTC').tz_convert('UTC'))
            time_in_position = time_in_position.days

            if self.position.is_long:
                if self.daily_hh or time_in_position > 5:
                    self.position.close()

        else:
            if self.daily_ll and actual_close > self.daily_sma_200:
                self.buy(size=self.risk/100)

bt_train = Backtest(
    train_data, 
    LowerLow,
    commission=0.0002, 
    cash=15_000, 
    margin=1/30
)

stats = bt_train.run()

bt_train.plot(filename='./RsiBBands.html', resample=False)


stats

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2014-01-01 12:00:00
End                       2015-12-31 17:00:00
Duration                    729 days 05:00:00
Exposure Time [%]                   15.708289
Equity Final [$]                  15288.41382
Equity Peak [$]                  15510.831654
Return [%]                           1.922759
Buy & Hold Return [%]               11.321139
Return (Ann.) [%]                    0.859171
Volatility (Ann.) [%]                3.415666
Sharpe Ratio                         0.251538
Sortino Ratio                        0.384912
Calmar Ratio                         0.231002
Max. Drawdown [%]                   -3.719317
Avg. Drawdown [%]                    -0.53521
Max. Drawdown Duration      153 days 05:00:00
Avg. Drawdown Duration       15 days 12:00:00
# Trades                                   20
Win Rate [%]                             50.0
Best Trade [%]                       4.530256
Worst Trade [%]                     -3.010582
Avg. Trade [%]                    

In [18]:
bt_train = Backtest(
    test_data, 
    LowerLow,
    commission=0.0002, 
    cash=15_000, 
    margin=1/30
)

stats = bt_train.run()

bt_train.plot(filename='./RsiBBands.html', resample=False)

stats

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2015-01-02 00:00:00
End                       2017-12-29 20:00:00
Duration                   1092 days 20:00:00
Exposure Time [%]                   14.832059
Equity Final [$]                 15544.652851
Equity Peak [$]                  15562.505851
Return [%]                           3.631019
Buy & Hold Return [%]               29.597007
Return (Ann.) [%]                    1.151543
Volatility (Ann.) [%]                3.032106
Sharpe Ratio                         0.379783
Sortino Ratio                        0.525429
Calmar Ratio                          0.30672
Max. Drawdown [%]                   -3.754381
Avg. Drawdown [%]                   -0.378311
Max. Drawdown Duration      267 days 07:00:00
Avg. Drawdown Duration       25 days 22:00:00
# Trades                                   30
Win Rate [%]                        66.666667
Best Trade [%]                       4.213227
Worst Trade [%]                     -3.454043
Avg. Trade [%]                    

# WFO

In [20]:
import pickle
from wfo_utils.utils import walk_forward, plot_full_equity_curve, plot_stats

In [28]:

lookback_bars = 365 * 24
validation_bars = 365 * 24
warmup_bars = 200 * 24 

params = {
    'n': [5, 7, 9, 10],
    'maximize': 'Return [%]' 
}


stats = walk_forward(
    LowerLow,
    wfo_data, 
    lookback_bars=lookback_bars,
    validation_bars=validation_bars,
    warmup_bars=warmup_bars, 
    params=params,
    commission=0.0002, 
    margin=1/30, 
    cash=15_000
)

train from 2014-01-01 12:00:00 to 2015-08-12 16:00:00


  0%|          | 0/4 [00:00<?, ?it/s]

validate from 2014-09-09 22:00:00 to 2017-11-23 14:00:00
equity final: 15627.485962800009
train from 2015-08-12 17:00:00 to 2017-11-23 14:00:00


  0%|          | 0/4 [00:00<?, ?it/s]

validate from 2016-08-23 09:00:00 to 2019-07-16 03:00:00
equity final: 15814.071171000014
train from 2017-11-23 15:00:00 to 2019-07-16 03:00:00


  0%|          | 0/4 [00:00<?, ?it/s]

validate from 2018-09-19 19:00:00 to 2021-01-11 04:00:00
equity final: 15872.899876200023
train from 2019-07-16 04:00:00 to 2021-01-11 04:00:00


  0%|          | 0/4 [00:00<?, ?it/s]

validate from 2020-03-18 04:00:00 to 2022-07-04 04:00:00
equity final: 16801.87449020003


In [29]:
for x in range(0, len(stats)):
    plot_stats(df, stats[x], LowerLow, plot=True)

Start                     2014-09-09 22:00:00
End                       2017-11-23 14:00:00
Duration                   1170 days 16:00:00
Exposure Time [%]                    14.60177
Equity Final [$]                 15627.485963
Equity Peak [$]                  15645.338963
Return [%]                            4.18324
Buy & Hold Return [%]               30.551363
Return (Ann.) [%]                    1.212335
Volatility (Ann.) [%]                2.974767
Sharpe Ratio                         0.407539
Sortino Ratio                        0.572386
Calmar Ratio                         0.304073
Max. Drawdown [%]                   -3.986989
Avg. Drawdown [%]                   -0.368349
Max. Drawdown Duration      403 days 07:00:00
Avg. Drawdown Duration       24 days 09:00:00
# Trades                                   33
Win Rate [%]                        66.666667
Best Trade [%]                       4.213227
Worst Trade [%]                     -3.454043
Avg. Trade [%]                    

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2016-08-23 09:00:00
End                       2019-07-16 03:00:00
Duration                   1056 days 18:00:00
Exposure Time [%]                   18.724189
Equity Final [$]                 15814.071171
Equity Peak [$]                  16143.118128
Return [%]                           1.193955
Buy & Hold Return [%]               37.798789
Return (Ann.) [%]                    0.367662
Volatility (Ann.) [%]                3.509392
Sharpe Ratio                         0.104765
Sortino Ratio                        0.135219
Calmar Ratio                         0.076099
Max. Drawdown [%]                   -4.831338
Avg. Drawdown [%]                   -0.288862
Max. Drawdown Duration      291 days 11:00:00
Avg. Drawdown Duration       16 days 07:00:00
# Trades                                   37
Win Rate [%]                        81.081081
Best Trade [%]                       1.782245
Worst Trade [%]                     -6.785661
Avg. Trade [%]                    

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2018-09-19 19:00:00
End                       2021-01-11 04:00:00
Duration                    844 days 09:00:00
Exposure Time [%]                   17.477876
Equity Final [$]                 15872.899876
Equity Peak [$]                  16303.700825
Return [%]                           0.372002
Buy & Hold Return [%]               30.697428
Return (Ann.) [%]                    0.129863
Volatility (Ann.) [%]                4.973979
Sharpe Ratio                         0.026109
Sortino Ratio                        0.036582
Calmar Ratio                         0.015754
Max. Drawdown [%]                   -8.243408
Avg. Drawdown [%]                    -0.76319
Max. Drawdown Duration      346 days 00:00:00
Avg. Drawdown Duration       21 days 18:00:00
# Trades                                   28
Win Rate [%]                        60.714286
Best Trade [%]                       2.904373
Worst Trade [%]                     -8.279673
Avg. Trade [%]                    

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2020-03-18 04:00:00
End                       2022-07-04 04:00:00
Duration                    838 days 00:00:00
Exposure Time [%]                   14.756637
Equity Final [$]                  16801.87449
Equity Peak [$]                  17666.990044
Return [%]                           5.852583
Buy & Hold Return [%]               57.409338
Return (Ann.) [%]                    2.027715
Volatility (Ann.) [%]                2.969936
Sharpe Ratio                         0.682747
Sortino Ratio                        1.036896
Calmar Ratio                         0.413998
Max. Drawdown [%]                   -4.897889
Avg. Drawdown [%]                   -0.336174
Max. Drawdown Duration      167 days 17:00:00
Avg. Drawdown Duration        5 days 06:00:00
# Trades                                   23
Win Rate [%]                        69.565217
Best Trade [%]                       2.973507
Worst Trade [%]                     -3.657767
Avg. Trade [%]                    

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


In [30]:
plot_full_equity_curve(
    df, 
    stats, 
    warmup_bars=warmup_bars,
    lookback_bars=lookback_bars, 
    overlay_price=True)

In [ ]:
trades = test_stats._trades.groupby(by=['ExitTime']).agg({'PnL':['sum','count'], 'Duration':'max'})
trades.columns = trades.columns.droplevel(0)
trades = trades.reset_index().rename(columns={'count':'ammount_trades'})
trades = trades.rename(columns={'sum':'profit'})
trades = trades.rename(columns={'max':'minutes_in_trade'})
trades

In [ ]:
trades.minutes_in_trade.describe()
